In [ ]:
!pip install praw pandas
!pip install spacy
!python -m spacy download en_core_web_sm

import praw
import pandas as pd
from google.colab import files
import spacy
import string
import logging
import time

# Suppress only PRAW's async environment warning
logging.getLogger("praw").setLevel(logging.ERROR)

# Load SpaCy English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])

# Create a translation table to remove punctuation
punct_table = str.maketrans('', '', string.punctuation)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 60.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Reddit API credentials
reddit = praw.Reddit(
    client_id="moeBipGvMV2JV1zD8cbQug",
    client_secret="NOvEicJPduFPhWcWzlRURnD9u-gmKQ",
    user_agent="Political_Polarization_Classifier by u/Typical-Section6888"
)

In [ ]:
def get_comments(subreddit_name, label, limit=20000):
    subreddit = reddit.subreddit(subreddit_name)
    comments = []

    for submission in subreddit.top(limit=300, time_filter='all'):  # Loop through top posts
        submission.comments.replace_more(limit=10)
        for comment in submission.comments.list():
            if len(comments) >= limit:
                break
            if comment.body and len(comment.body) > 10:  # Ignore very short or deleted
                comments.append({
                    'text': comment.body,
                    'label': label
                })
        if len(comments) >= limit:
            break
    return comments

In [ ]:
def preprocess_text(text):
    # Lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(punct_table)

    # Tokenize and lemmatize with stopword removal
    doc = nlp(text)
    tokens = [
        token.lemma_ for token in doc
        if not token.is_stop and not token.is_punct and not token.is_space
    ]

    return " ".join(tokens)

In [ ]:
# Scrape
time.sleep(10)
democrat_comments = get_comments('Democrats', label=1)
time.sleep(20)
liberal_comments = get_comments('Liberal', label=1)
time.sleep(20)
republican_comments = get_comments('Republican', label=0)
time.sleep(20)
conservative_comments = get_comments('Conservative', label=0)
time.sleep(10)

# Combine and clean
all_comments = democrat_comments + liberal_comments + republican_comments + conservative_comments
df = pd.DataFrame(all_comments)
df = df.sample(frac=1).reset_index(drop=True)  # Shuffle
df['clean_text'] = df['text'].apply(preprocess_text)

# Export
df[['clean_text', 'label']].to_csv('reddit_comments_clean.csv', index=False)
files.download('reddit_comments_clean.csv')
print(f"✅ Saved {len(df)} comments to reddit_comments_clean.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Saved 79300 comments to reddit_comments_clean.csv
